In [1]:
import cptac
import pandas as pd
import cptac.utils as ut

In [2]:
luad = cptac.Luad()
lscc = cptac.Lscc()
hnscc = cptac.Hnscc()
en = cptac.Endometrial()
ccrcc = cptac.Ccrcc()

cptac warning: The LSCC data is currently strictly reserved for CPTAC investigators. Otherwise, you are not authorized to access these data. Additionally, even after these data become publicly available, they will be subject to a publication embargo (see https://proteomics.cancer.gov/data-portal/about/data-use-agreement or enter cptac.embargo() to open the webpage for more details). (<ipython-input-2-5f7029bc4773>, line 2)


In [3]:
cancer_list = [luad, lscc, hnscc, en, ccrcc]

In [4]:
cancer_dict = {lscc: 'lscc', luad: 'luad', hnscc: 'hnscc', en: 'en', ccrcc: 'ccrcc'}

In [5]:
mutation_frequency_dfs = []
for cancer in cancer_list:
    mutation_df = cancer.get_somatic_mutation()
    mutation_df = mutation_df[mutation_df.Mutation != 'Silent']
    mutation_df = mutation_df[mutation_df.Mutation != 'RNA']
    gene_df = cancer.get_transcriptomics('tumor')
    if isinstance(gene_df.columns, pd.MultiIndex):
        gene_df = gene_df.droplevel('Database_ID', axis = 1)
    mutation_frequency = []
    genes = list(pd.unique(mutation_df.Gene))
    for gene in genes:
        num_mutated = len(pd.unique(mutation_df[mutation_df.Gene == gene].index))
        total = len(pd.unique(gene_df.index))
        frequency = num_mutated / total 
        mutation_frequency.append(frequency)
    df = pd.DataFrame({'Gene': genes, 'Mutation_Frequency': mutation_frequency})
    df = df.sort_values(by= 'Mutation_Frequency', ascending = False)
    if len(df) > 10:
        df = df[0:10]
    df['cancer'] = [cancer_dict[cancer]] * len(df)
    mutation_frequency_dfs.append(df)
mutation_frequency_df = pd.concat(mutation_frequency_dfs)
mutation_frequency_df

,Gene,Mutation_Frequency,cancer
263,TP53,0.536364,luad
871,EGFR,0.345455,luad
1020,MUC16,0.336364,luad
225,RYR2,0.318182,luad
40,TTN,0.318182,luad
153,LRP1B,0.309091,luad
1010,KRAS,0.300000,luad
1221,CSMD3,0.300000,luad
273,USH2A,0.254545,luad
281,ZFHX4,0.236364,luad


In [7]:
# write bash command for finding permutation p-val for each gene/cancer pair
with open('transmutation_effects_permutations.sh', 'w') as file:
    for index, row in mutation_frequency_df.iterrows():
        s = 'python supercomputer_transmutation_effects.py ' + row.cancer + ' ' + row.Gene + ' 10000\n'
        file.write(s)
    file.close()
    